# Cryptocurrency data analysis and price prediction using ARIMA and Seasonal ARIMA model
#### Project Proposal

## 1. Aims, objective and background
### 1.1 Introduction
Cryptocurrencies are quickly challenging traditional currencies throughout the world. Digital currencies may be purchased in a variety of ways, making them accessible to everyone, and merchants adopting multiple cryptocurrencies could signal that money as we know it is about to undergo a huge transformation.

Bitcoin was the first cryptocurrency launched in the year 2009 and since then the popularity and acceptibility of blockchain and cryptocurrencies has only grown upward direction. Due to its unique qualities of blockchain, such as security and transparency, cheaper cost and decentralisation, it is already being used to solve variety of real world problems as per Mallqui and Fernandes [5].

This project was inspired by Chaudhari, A. (2020, June 11). Forecasting Cryptocurrency Prices using Machine Learning [3] and Chakrabarti, S. (2021, December 3). Cryptocurrency Price Prediction using ARIMA Model [4]. The authors used data science ARIMA, LSTM and Prophet models to predict the cryptocurrency prices.

I have decided to focus on next few months price prediction of 4 cryptocurrencies which includes Bitcoin (BTC), Ethereum (ETH), Shiba Inu (SHIB), Solana (SOL).

### 1.2 Aims and objectives
For this project I would like to exlpore following things
1. Analysing of history data of cryptocurrency.
2. Using history data making visualizations (graphs)
3. Looking for relationship of the graphs.
4. Checking the instantaneous move of one coin (BTC) affect others.
5. Using (Autoregressive integrated moving average) model and SARIMAX (Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors) model for predicting next months prices.

For this project my aims are
1. Decide how much history data is required to use ARIMA and SARIMAX models.
2. Decide which currencies will be best for my project.
3. Collecting data via API/wesrapping and storing in better format so that I can do data cleaning.
4. Clean the scraped or API history data suitable for data analysis
5. Perform some data analysis to see if there are any trends in the data that may be helpful for future investigation.
6. Calculate the accuracy of price predication.


## 7. References and Resources
### 7.1 References
[1]: Use our data. (n.d.). Investing Support. https://www.investing-support.com/hc/en-us/articles/360002357417
[2]: FRANKENFIELD, J. A. K. E. (2021, December 20). Cryptocurrency. Investopedia. https://www.investopedia.com/terms/c/cryptocurrency.asp
[3]: Chaudhari, A. (2020, June 11). Forecasting Cryptocurrency Prices using Machine Learning - NORMA@NCI Library. Norma.Ncirl.Ie. http://norma.ncirl.ie/4272/
[4]: Chakrabarti, S. (2021, December 3). Cryptocurrency Price Prediction using ARIMA Model. Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/12/cryptocurrency-price-prediction-using-arima-model/
[5]: Mallqui, D., & Fernandes, R. (2019b, February 1). Predicting the direction, maximum, minimum and closing prices of daily Bitcoin exchange rate using machine learning techniques. ScienceDirect. https://www.sciencedirect.com/science/article/pii/S1568494618306707
[6]: George, D. (2021, December 14). A Brief Introduction to ARIMA and SARIMAX Modeling in Python. Medium. https://medium.com/swlh/a-brief-introduction-to-arima-and-sarima-modeling-in-python-87a58d375def.


### 7.2 Resources used